In [ ]:
import requests
import urllib
import os
from datetime import datetime
import zipfile
import pandas as pd
import numpy as np
    

In [ ]:
filepath = r"C:\Users\ljerome\Box Sync\My Personal Workspace\Bike Share"

Download the zip files and extract the files

In [ ]:
year = 2011 #2011 is first full year of data
quarters = ['Q1','Q2','Q3','Q4']
while year<=datetime.now().year:#urllib.request.urlopen(url).status == 200:
    for quarter in quarters:
        if year<2016:
            url = "https://s3.amazonaws.com/capitalbikeshare-data/" + str(year) + "-" + quarter + "-cabi-trip-history-data.zip"
            #print(url)
        elif year == 2016 and quarter == 'Q1' :
            url = "https://s3.amazonaws.com/capitalbikeshare-data/" + str(year) + "-" + quarter + "-cabi-trip-history-data.zip"
            #print(url)
        else:
            url = "https://s3.amazonaws.com/capitalbikeshare-data/" + str(year) + "-" + quarter + "-cabi-trips-history-data.zip"
            #print(url)       
        r = requests.get(url)
        filename = "CBS_"+str(year)+quarter+".zip"
        print(filename)
        with open(filename, "wb") as code:
           code.write(r.content)
        zip_ref = zipfile.ZipFile(filename, 'r')
        zip_ref.extractall()
        zip_ref.close()

    year+=1

Pull out the csv files

In [ ]:
all_files = os.listdir()
csv_files = [cur_file for cur_file in all_files if cur_file.find(".csv")!=-1]

create a dictionary holding the dataframes key = file name value = dataframe

In [ ]:
df_dict = {}
for csv_file in csv_files:
    #print(csv_file[0:7])
    df_dict[csv_file] = pd.read_csv(csv_file)

In [7]:
key_list = list(df_dict.keys())

In [85]:
for key in key_list:
    df_dict[key].columns = [column.lower() for column in df_dict[key].columns]
    columns = df_dict[key].columns
    if 'end station number' not in columns:
        df_dict[key]['end station number'] = None
    if 'start station number' not in columns:
        df_dict[key]['start station number'] = None
    #if 'Duration (ms)' or 'Total duration (ms)' in columns:
    df_dict[key] = df_dict[key].rename(index = str,columns={'duration (ms)':'Duration'})
    df_dict[key] = df_dict[key].rename(columns={'total duration (ms)':'Duration'})
    #if 'Bike#' or 'Bike #' in columns:
    df_dict[key] = df_dict[key].rename(columns={'bike#':'bike number'})
    df_dict[key] = df_dict[key].rename(columns={'bike #':'bike number'})
    #if 'Member type' or 'Type' or 'Bike Key' in columns:
    df_dict[key] = df_dict[key].rename(columns={'member type':'subscription type'})
    df_dict[key] = df_dict[key].rename(columns={'type':'subscription type'})
    df_dict[key] = df_dict[key].rename(columns={'bike key':'subscription type'})
    df_dict[key] = df_dict[key].rename(columns={'subscriber type':'subscription type'})
    df_dict[key] = df_dict[key].rename(columns={'account type':'subscription type'})
    df_dict[key] = df_dict[key].rename(columns={'start time':'start date'})



In [86]:
all_data = pd.DataFrame()
for key in key_list:
    all_data = all_data.append(df_dict[key])

In [87]:
def convert_to_ms(duration):
    time_in_ms = 0
    if type(duration) == str:
        time_part = duration.split(" ")
        time_in_ms+=int(time_part[0].replace('h',""))*60*60*1000
        try:
            time_in_ms+=int(time_part[1].replace('min.',""))*60*1000
        except ValueError:
            time_in_ms+=int(time_part[1].replace('m',""))*60*1000
        try:
            time_in_ms+=int(time_part[2].replace('sec.',""))*1000
        except ValueError:
            time_in_ms+=int(time_part[2].replace('s',""))*1000
    else:
        time_in_ms = duration
    return time_in_ms

In [88]:
def convert_station_name(station):
    if type(station) != float:
        loc_int = station.find("(")
        if loc_int != -1:
            converted_station_name = station[:loc_int-1]
        else:
            return station
    else:
        return station
    return converted_station_name

In [89]:
all_data['duration']=all_data['duration'].apply(convert_to_ms)

In [90]:
all_data['start station']=all_data['start station'].apply(convert_station_name)
all_data['end station']=all_data['end station'].apply(convert_station_name)

In [91]:
r = requests.get('https://gbfs.capitalbikeshare.com/gbfs/en/station_information.json')
data = r.json()


In [92]:
df_stations = pd.DataFrame(data['data']['stations'])
df_stations.head()

,capacity,eightd_has_key_dispenser,lat,lon,name,region_id,rental_methods,short_name,station_id
0,15,False,38.858971,-77.053230,Eads St & 15th St S,41,"[KEY, CREDITCARD]",31000,1
1,11,False,38.857250,-77.053320,18th & Eads St.,41,"[KEY, CREDITCARD]",31001,2
2,17,False,38.856425,-77.049232,20th & Crystal Dr,41,"[KEY, CREDITCARD]",31002,3
3,11,False,38.860170,-77.049593,15th & Crystal Dr,41,"[KEY, CREDITCARD]",31003,4
4,11,False,38.857866,-77.059490,Aurora Hills Community Ctr/18th & Hayes St,41,"[KEY, CREDITCARD]",31004,5


In [93]:
all_data[all_data['start station number'].notnull()]

,bike number,duration,end date,end station,end station number,start date,start station,start station number,subscription type
0,W21516,257866,7/1/2015 0:04,15th & Euclid St NW,31117.0,7/1/2015 0:00,California St & Florida Ave NW,31116.0,Registered
1,W00476,2683934,7/1/2015 0:45,Jefferson Dr & 14th St SW,31247.0,7/1/2015 0:00,Jefferson Dr & 14th St SW,31247.0,Casual
2,W20384,2349547,7/1/2015 0:41,Jefferson Dr & 14th St SW,31247.0,7/1/2015 0:02,Jefferson Dr & 14th St SW,31247.0,Casual
3,W00972,826571,7/1/2015 0:16,Park Rd & Holmead Pl NW,31602.0,7/1/2015 0:02,New Hampshire Ave & T St NW,31229.0,Registered
4,W21565,123046,7/1/2015 0:06,3rd St & Pennsylvania Ave SE,31269.0,7/1/2015 0:04,Eastern Market / 7th & North Carolina Ave SE,31610.0,Registered
5,W21688,2176084,7/1/2015 0:42,M St & New Jersey Ave SE,31208.0,7/1/2015 0:05,Lincoln Memorial,31258.0,Casual
6,W21741,2652620,7/1/2015 0:50,13th St & New York Ave NW,31227.0,7/1/2015 0:06,New York Ave & 15th St NW,31222.0,Casual
7,W21572,1045864,7/1/2015 0:25,N Quincy St & Glebe Rd,31035.0,7/1/2015 0:08,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,Registered
8,W00700,2481165,7/1/2015 0:50,13th St & New York Ave NW,31227.0,7/1/2015 0:08,New York Ave & 15th St NW,31222.0,Casual
9,W20468,114563,7/1/2015 0:10,Eastern Market Metro / Pennsylvania Ave & 7th ...,31613.0,7/1/2015 0:08,Eastern Market / 7th & North Carolina Ave SE,31610.0,Registered


In [81]:
all_data_with_stations_on_start_number = all_data[all_data['start station number']].merge(df_stations,how='left',left_on='start station number',right_on='short_name')
all_data_with_stations_on_end_number = all_data[all_data['end station number']].merge(df_stations,how='left',left_on='end station number',right_on='short_name')

ValueError: cannot index with vector containing NA / NaN values

In [20]:
#all_data_with_stations_on_start_address = all_data[all_data['start station number'].isnull()].merge(df_stations,how='left',left_on='start station',right_on='ADDRESS')

In [21]:
all_data_with_loc = pd.DataFrame()

In [22]:
all_data_with_loc = all_data_with_stations_on_start_number.append(all_data_with_stations_on_start_address)

In [23]:
all_data_with_loc.to_csv(r"C:\Users\ljerome\Box Sync\My Personal Workspace\Bike Share\All_CBS_Data\cbs.csv")

In [35]:
df_dict['2015-Q3-cabi-trip-history-data.csv'][df_dict['2015-Q3-cabi-trip-history-data.csv']['start date'] == '7/1/2015 7:23']

,Duration,start date,end date,start station number,start station,end station number,end station,bike number,subscription type
687,589800,7/1/2015 7:23,7/1/2015 7:32,31251,12th & L St NW,31217,USDA / 12th & Independence Ave SW,W00204,Registered
688,526863,7/1/2015 7:23,7/1/2015 7:31,31105,14th & Harvard St NW,31201,15th & P St NW,W22060,Registered
690,466444,7/1/2015 7:23,7/1/2015 7:30,31619,Lincoln Park / 13th & East Capitol St NE,31271,Constitution Ave & 2nd St NW/DOL,W21423,Registered
691,531714,7/1/2015 7:23,7/1/2015 7:32,31212,21st & M St NW,31231,14th & D St NW / Ronald Reagan Building,W22093,Registered
692,689808,7/1/2015 7:23,7/1/2015 7:34,31121,Calvert St & Woodley Pl NW,31250,20th & L St NW,W21939,Registered
693,1021850,7/1/2015 7:23,7/1/2015 7:40,31118,3rd & Elm St NW,31212,21st & M St NW,W20053,Registered
694,837156,7/1/2015 7:23,7/1/2015 7:37,31102,11th & Kenyon St NW,31275,New Hampshire Ave & 24th St NW,W00087,Registered
695,198630,7/1/2015 7:23,7/1/2015 7:26,31212,21st & M St NW,31257,22nd & I St NW / Foggy Bottom,W01278,Registered
696,712740,7/1/2015 7:23,7/1/2015 7:35,31709,34th St & Minnesota Ave SE,31709,34th St & Minnesota Ave SE,W00088,Registered
697,366032,7/1/2015 7:23,7/1/2015 7:30,31614,11th & H St NE,31623,Columbus Circle / Union Station,W20349,Registered


In [39]:
df_stations[df_stations['TERMINAL_NUMBER']==31709]

,OBJECTID,ID,ADDRESS,TERMINAL_NUMBER,LATITUDE,LONGITUDE
